In [1]:
import os
import re

import pystac
import pystac.item
import pystac.link
from pystac.provider import ProviderRole
from pystac.extensions.projection import ProjectionExtension
from pystac.extensions.item_assets import AssetDefinition

from pyproj import Transformer
from shapely.geometry import GeometryCollection, box, shape, mapping
from datetime import datetime, UTC

import rasterio as rio
import rasterio.warp
import rasterio.crs

import xml.etree.cElementTree as ET
from xml.dom import minidom

In [2]:
# img_path = 'X:/EO/u2018_clc2012_v2020_20u1_raster100m/DATA/U2018_CLC2012_V2020_20u1.tif'
# os.path.split(img_path)

In [43]:
# This goes into constants.py ...

STAC_DIR = 'stac_tests'
COLLECTION_ID = 'corine-land-cover-raster'

DOM_DICT = {
    'GLP': 'Guadeloupe',
    'GUF': 'French Guyana',
    'MTQ': 'Martinique',
    'MYT': 'Mayotte',
    'REU': 'Réunion',
    '': 'Europe',
}

MEDIA_TYPE_DICT = {
    'tif': pystac.MediaType.COG,
    'tif_xml': pystac.MediaType.XML,
    'tif_aux_xml': pystac.MediaType.XML,
    'tif_ovr': 'image/tiff; application=geotiff; profile=pyramid',
    'tif_vat_cpg': pystac.MediaType.TEXT,
    'tif_vat_dbf': 'application/dbf',
    'txt': pystac.MediaType.TEXT,
    'tif_lyr': 'image/tiff; application=geotiff; profile=layer',
    'tfw': pystac.MediaType.TEXT,
    'xml': pystac.MediaType.XML,
}

ROLES_DICT = {
    'tif': ['data', 'visual'],
    'tif_xml': ['metadata'],
    'tif_aux_xml': ['metadata'],
    'tif_ovr': ['metadata'],
    'tif_vat_cpg': ['metadata'],
    'tif_vat_dbf': ['metadata'],
    'txt': ['metadata'],
    'tif_lyr': ['metadata'],
    'tfw': ['metadata'],
    'xml': ['metadata'],
}

TITLE_DICT = {
    'tif': 'Single Band Land Classification {label}',
    'tif_xml': 'TIFF Metadata {label}',
    'tif_aux_xml': 'TIFF Statistics {label}',
    'tif_ovr': 'Pyramid {label}',
    'tif_vat_cpg': 'Encoding {label}',
    'tif_vat_dbf': 'Database {label}',
    'txt': 'Legends {label}',
    'tif_lyr': 'Legend Layer {label}',
    'tfw': 'World File {label}',
    'xml': 'Single Band Land Classification Metadata {label}',
}

CLC_PROVIDER = pystac.provider.Provider(
    name='Copernicus Land Monitoring Service',
    description=('The Copernicus Land Monitoring Service provides '
                'geographical information on land cover and its '
                'changes, land use, ground motions, vegetation state, '
                'water cycle and Earth\'s surface energy variables to '
                'a broad range of users in Europe and across the World '
                'in the field of environmental terrestrial applications.'),
    roles=[ProviderRole.LICENSOR, ProviderRole.HOST],
    url='https://land.copernicus.eu'
)


ITEM_DESCRIPTION = ('Corine Land Cover {year} (CLC{year}) is one of the Corine Land Cover (CLC) ' 
                    'datasets produced within the frame the Copernicus Land Monitoring Service '
                    'referring to land cover / land use status of year {year}. '
                    'CLC service has a long-time heritage (formerly known as \"CORINE Land Cover Programme\"), '
                    'coordinated by the European Environment Agency (EEA). It provides consistent '
                    'and thematically detailed information on land cover and land cover changes across Europe. '
                    'CLC datasets are based on the classification of satellite images produced by the national '
                    'teams of the participating countries - the EEA members and cooperating countries (EEA39). '
                    'National CLC inventories are then further integrated into a seamless land cover map of Europe. '
                    'The resulting European database relies on standard methodology and nomenclature with following '
                    'base parameters: 44 classes in the hierarchical 3-level CLC nomenclature; '
                    'minimum mapping unit (MMU) for status layers is 25 hectares; '
                    'minimum width of linear elements is 100 metres. '
                    'Change layers have higher resolution, i.e. minimum mapping unit (MMU) is 5 hectares '
                    'for Land Cover Changes (LCC), and the minimum width of linear elements is 100 metres. '
                    'The CLC service delivers important data sets supporting the implementation of key priority '
                    'areas of the Environment Action Programmes of the European Union as e.g. protecting ecosystems, '
                    'halting the loss of biological diversity, tracking the impacts of climate change, '
                    'monitoring urban land take, assessing developments in agriculture or dealing with '
                    'water resources directives. CLC belongs to the Pan-European component of the '
                    'Copernicus Land Monitoring Service (https://land.copernicus.eu/), part of the '
                    'European Copernicus Programme coordinated by the European Environment Agency, '
                    'providing environmental information from a combination of air- and space-based observation '
                    'systems and in-situ monitoring. Additional information about CLC product description including '
                    'mapping guides can be found at https://land.copernicus.eu/user-corner/technical-library/. '
                    'CLC class descriptions can be found at '
                    'https://land.copernicus.eu/user-corner/technical-library/corine-land-cover-nomenclature-guidelines/html/.')


In [44]:
'''
def deconstruct_clc_name(filename: str):
    id = os.path.basename(filename).split('.')[0]
    p = re.compile(("U(?P<update_campaign>[0-9]{4})_"
                    "(?P<theme>CLC|CHA)(?P<reference_year>[0-9]{4})_"
                    "V(?P<release_year>[0-9]{4})_(?P<release_number>[0-9a-z]*)"
                    "_?(?P<country_code>[A-Z]*)?"
                    "_?(?P<DOM_code>[A-Z]*)?"))
    m = p.search(id)

    return(m.groupdict())
'''

def deconstruct_clc_name(filename: str):
    p = re.compile('^(?P<id>[A-Z0-9a-z_]*).(?P<suffix>.*)$')
    m = p.search(os.path.basename(filename))

    filename_split = m.groupdict()

    p = re.compile(("U(?P<update_campaign>[0-9]{4})_"
                    "(?P<theme>CLC|CHA)(?P<reference_year>[0-9]{4})_"
                    "V(?P<release_year>[0-9]{4})_(?P<release_number>[0-9a-z]*)"
                    "_?(?P<country_code>[A-Z]*)?"
                    "_?(?P<DOM_code>[A-Z]*)?"))
    m = p.search(filename_split['id'])
    
    if m:
        return(m.groupdict() | filename_split)
    else:
        return(filename_split)


def create_asset(filename: str, DOM_code: str):
    filename_elements = deconstruct_clc_name(filename)
    suffix = filename_elements['suffix'].replace('.', '_')
    
    label = DOM_DICT[DOM_code]
    
    asset = pystac.Asset(href=filename, title=TITLE_DICT[suffix].format(label=label), media_type=MEDIA_TYPE_DICT[suffix], roles=ROLES_DICT[suffix])
    return(f"{filename_elements['id']}_{suffix}", asset)

def get_img_paths(path: str):    
    img_paths=[]
    for root, dirs, files in os.walk(path):
        if root.endswith(('DATA', 'French_DOMs')):
            for file in files:
                if file.endswith('.tif'):
                    img_paths.append(os.path.join(root, file))

    return(img_paths)

# Not needed anymore, part of deconstruct_clc_name...
# def extract_clc_name(path: str):
#     clc_name = os.path.basename(path).split('.')[0]
#     return(clc_name)

def get_asset_files(path, clc_name):

    clc_name_elements = deconstruct_clc_name(clc_name)

    asset_files = []
    
    for root, dirs, files in os.walk(path):
        if not clc_name_elements['DOM_code'] and 'French_DOMs' in root:
            continue
        
        if clc_name_elements['DOM_code'] and ('Legend' in root and not 'French_DOMs' in root):
            continue
        
        for file in files:
            if (file.startswith(clc_name + '.') or 
                file.endswith((f'{clc_name_elements["DOM_code"]}.tif.lyr', 'QGIS.txt',)) and 
                clc_name in file):
                asset_files.append(os.path.join(root, file))

    return(asset_files)
 
def project_bbox(img, target_epsg=4326):
    target_crs = rio.crs.CRS.from_epsg(target_epsg)
    bbox_warped = rio.warp.transform_bounds(img.crs, target_crs, *img.bounds)
    return(bbox_warped)

def create_item(img_path, root):

    clc_name_elements = deconstruct_clc_name(img_path)

    asset_files = get_asset_files(root, clc_name=clc_name_elements['id'])
    asset_files = [f for f in asset_files if not f.endswith('aux')]
    year = clc_name_elements.get('reference_year')
    props = {'description': ITEM_DESCRIPTION.format(year=year),
            'created': None,
            'providers': CLC_PROVIDER.to_dict(),
    }

    with rio.open(img_path) as img:

        bbox = project_bbox(img)
        params = {
            'id': clc_name_elements.get('id'),
            'bbox': bbox,
            'geometry': mapping(box(*bbox)),
            'datetime': None,
            'start_datetime': datetime(int(year), 1, 1, microsecond=0, tzinfo=UTC),
            'end_datetime': datetime(int(year), 12, 31, microsecond=0, tzinfo=UTC),
            'properties': props,
        }

    item = pystac.Item(**params)
    
    for asset_file in asset_files:
        # print(asset_file)
        key, asset = create_asset(asset_file, DOM_code=clc_name_elements.get('DOM_code'))
        item.add_asset(
            key=key,
            asset=asset,
        )

    proj_ext = ProjectionExtension.ext(item.assets[os.path.basename(img_path).replace('.', '_')], add_if_missing=True)
    proj_ext.apply(epsg=rio.crs.CRS(img.crs).to_epsg(),
                   bbox=img.bounds,
                   shape=[_ for _ in img.shape],
                   transform=[_ for _ in img.transform] + [0.0, 0.0, 1.0],
                   )

    license = pystac.link.Link(rel='LICENSE', target="https://land.copernicus.eu/en/data-policy")
    item.add_link(license)

    return(item)

In [45]:
os.getcwd()

'x:\\projects\\ETC-DI\\Task_18\\clms-stac\\scripts\\clc'

In [48]:
root = '../../../CLC_100m'

img_paths = get_img_paths(path=root)
print(img_paths)
# for img_path in img_paths:

img_path = img_paths[3]

get_asset_files(root, deconstruct_clc_name(img_path)['id'])

['../../../CLC_100m\\u2018_clc2012_v2020_20u1_raster100m\\DATA\\U2018_CLC2012_V2020_20u1.tif', '../../../CLC_100m\\u2018_clc2012_v2020_20u1_raster100m\\DATA\\French_DOMs\\U2018_CLC2012_V2020_20u1_FR_GLP.tif', '../../../CLC_100m\\u2018_clc2012_v2020_20u1_raster100m\\DATA\\French_DOMs\\U2018_CLC2012_V2020_20u1_FR_GUF.tif', '../../../CLC_100m\\u2018_clc2012_v2020_20u1_raster100m\\DATA\\French_DOMs\\U2018_CLC2012_V2020_20u1_FR_MTQ.tif', '../../../CLC_100m\\u2018_clc2012_v2020_20u1_raster100m\\DATA\\French_DOMs\\U2018_CLC2012_V2020_20u1_FR_MYT.tif', '../../../CLC_100m\\u2018_clc2012_v2020_20u1_raster100m\\DATA\\French_DOMs\\U2018_CLC2012_V2020_20u1_FR_REU.tif', '../../../CLC_100m\\u2018_clc2018_v2020_20u1_raster100m\\DATA\\U2018_CLC2018_V2020_20u1.tif', '../../../CLC_100m\\u2018_clc2018_v2020_20u1_raster100m\\DATA\\French_DOMs\\U2018_CLC2018_V2020_20u1_FR_GLP.tif', '../../../CLC_100m\\u2018_clc2018_v2020_20u1_raster100m\\DATA\\French_DOMs\\U2018_CLC2018_V2020_20u1_FR_GUF.tif', '../../../CLC

['../../../CLC_100m\\u2018_clc2012_v2020_20u1_raster100m\\DATA\\French_DOMs\\U2018_CLC2012_V2020_20u1_FR_MTQ.tfw',
 '../../../CLC_100m\\u2018_clc2012_v2020_20u1_raster100m\\DATA\\French_DOMs\\U2018_CLC2012_V2020_20u1_FR_MTQ.tif',
 '../../../CLC_100m\\u2018_clc2012_v2020_20u1_raster100m\\DATA\\French_DOMs\\U2018_CLC2012_V2020_20u1_FR_MTQ.tif.aux.xml',
 '../../../CLC_100m\\u2018_clc2012_v2020_20u1_raster100m\\DATA\\French_DOMs\\U2018_CLC2012_V2020_20u1_FR_MTQ.tif.ovr',
 '../../../CLC_100m\\u2018_clc2012_v2020_20u1_raster100m\\DATA\\French_DOMs\\U2018_CLC2012_V2020_20u1_FR_MTQ.tif.vat.cpg',
 '../../../CLC_100m\\u2018_clc2012_v2020_20u1_raster100m\\DATA\\French_DOMs\\U2018_CLC2012_V2020_20u1_FR_MTQ.tif.vat.dbf',
 '../../../CLC_100m\\u2018_clc2012_v2020_20u1_raster100m\\DATA\\French_DOMs\\U2018_CLC2012_V2020_20u1_FR_MTQ.tif.xml',
 '../../../CLC_100m\\u2018_clc2012_v2020_20u1_raster100m\\Metadata\\U2018_CLC2012_V2020_20u1_FR_MTQ.xml']

In [50]:
item = create_item(img_path, root)
item

<Item id=U2018_CLC2012_V2020_20u1_FR_MTQ>

In [51]:
asset_def = AssetDefinition()
asset_def.item_assets(item)
item.make_asset_hrefs_relative()

TypeError: AssetDefinition.__init__() missing 1 required positional argument: 'properties'

In [52]:
WORKING_DIR = os.getcwd()

CLMS_CATALOG_LINK = pystac.link.Link(
    rel=pystac.RelType.ROOT, target=pystac.STACObject.from_file(os.path.join(WORKING_DIR, "stacs/clms_catalog.json"))
)
COLLECTION_LINK = pystac.link.Link(
    rel=pystac.RelType.COLLECTION,
    target=pystac.STACObject.from_file(os.path.join(WORKING_DIR, f"stacs/{COLLECTION_ID}/{COLLECTION_ID}.json")),
)
ITEM_PARENT_LINK = pystac.link.Link(
    rel=pystac.RelType.PARENT,
    target=pystac.STACObject.from_file(os.path.join(WORKING_DIR, f"stacs/{COLLECTION_ID}/{COLLECTION_ID}.json")),
)

item.set_self_href(os.path.join(WORKING_DIR, f"{STAC_DIR}/{COLLECTION_ID}/{item.id}/{item.id}.json"))
item.save_object()

FileNotFoundError: [Errno 2] No such file or directory: 'x:\\projects\\ETC-DI\\Task_18\\clms-stac\\scripts\\clc\\stacs/clms_catalog.json'

In [62]:
# # Taken from https://stackoverflow.com/questions/2148119/how-to-convert-an-xml-string-to-a-dictionary
# from xml.etree import cElementTree as ElementTree


# class XmlListConfig(list):
#     def __init__(self, aList):
#         for element in aList:
#             if element:
#                 if len(element) == 1 or element[0].tag != element[1].tag:
#                     self.append(XmlDictConfig(element))
#                 elif element[0].tag == element[1].tag:
#                     self.append(XmlListConfig(element))
#             elif element.text:
#                 text = element.text.strip()
#                 if text:
#                     self.append(text)


# class XmlDictConfig(dict):
#     def __init__(self, parent_element):
#         if parent_element.items():
#             self.update(dict(parent_element.items()))
#         for element in parent_element:
#             if element:
#                 if len(element) == 1 or element[0].tag != element[1].tag:
#                     aDict = XmlDictConfig(element)
#                 else:
#                     aDict = {element[0].tag: XmlListConfig(element)}
#                 if element.items():
#                     aDict.update(dict(element.items()))
#                 self.update({element.tag: aDict})
#             elif element.items():
#                 self.update({element.tag: dict(element.items())})
#             else:
#                 self.update({element.tag: element.text})

# stac_io = pystac.StacIO.default()

# def get_metadata(xml: str):
#     result = XmlDictConfig(ElementTree.XML(stac_io.read_text(xml)))
#     result[
#         "ORIGINAL_URL"
#     ] = xml  # Include the original URL in the metadata for use later
#     return result


# xml_path = '../CLC_samples/U2018_CLC2018_V2020_20u1.xml'

# get_metadata(xml_path)
